<a href="https://www.kaggle.com/code/elza10/titanic-survival-prediction?scriptVersionId=179021506" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


<h1>Uploading the data</h1> 

In [21]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
# gender_submission_data = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

<h1>Cleaning the data</h1>

In [22]:
train_data = train_data.drop(columns = ['Name','Ticket','Fare','Cabin'],axis = 1)
test_data = test_data.drop(columns = ['Name','Ticket','Fare','Cabin'],axis = 1)



In [23]:
test_data.head(100)


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Embarked
0,892,3,male,34.5,0,0,Q
1,893,3,female,47.0,1,0,S
2,894,2,male,62.0,0,0,Q
3,895,3,male,27.0,0,0,S
4,896,3,female,22.0,1,1,S
...,...,...,...,...,...,...,...
95,987,3,male,25.0,0,0,S
96,988,1,female,76.0,1,0,S
97,989,3,male,29.0,0,0,S
98,990,3,female,20.0,0,0,S


In [24]:
moy_age_Pclass1 = int((train_data[train_data.Pclass == 1]["Age"]).mean())
moy_age_Pclass2= int((train_data[train_data.Pclass == 2]["Age"]).mean())
moy_age_Pclass3= int((train_data[train_data.Pclass == 3]["Age"]).mean())


def age_depend_on_class(tab):
    age = tab.iloc[0]
    Pclass = tab.iloc[1]
    if pd.isnull(age):
        if Pclass == 1:
            age = moy_age_Pclass1
        elif Pclass == 2:      
            age = moy_age_Pclass2
        elif Pclass == 3:      
            age = moy_age_Pclass3
                                 
    return age

In [25]:
train_data['Age'] = train_data[['Age','Pclass']].apply(age_depend_on_class,axis = 1)
test_data['Age'] = test_data[['Age','Pclass']].apply(age_depend_on_class,axis = 1)

In [26]:
train_data.dropna(inplace = True)
test_data.dropna(inplace = True)

In [27]:
test_data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Embarked
0,892,3,male,34.5,0,0,Q
1,893,3,female,47.0,1,0,S
2,894,2,male,62.0,0,0,Q
3,895,3,male,27.0,0,0,S
4,896,3,female,22.0,1,1,S


In [28]:
train_sex = pd.get_dummies(train_data['Sex'], drop_first = True, dtype = int) 
train_embarked = pd.get_dummies(train_data['Embarked'], drop_first = True, dtype = int) 
test_sex = pd.get_dummies(test_data['Sex'], drop_first = True, dtype = int) 
test_embarked = pd.get_dummies(test_data['Embarked'], drop_first = True, dtype = int) 

In [29]:
test_sex.head()

,male
0,1
1,0
2,1
3,1
4,0


In [30]:
train_data.drop(['Sex','Embarked'],axis = 1, inplace = True)
test_data.drop(['Sex','Embarked'],axis = 1, inplace = True)
test_data.head()

,PassengerId,Pclass,Age,SibSp,Parch
0,892,3,34.5,0,0
1,893,3,47.0,1,0
2,894,2,62.0,0,0
3,895,3,27.0,0,0
4,896,3,22.0,1,1


In [31]:
train_data = pd.concat([train_data,train_sex,train_embarked], axis = 1)
test_data = pd.concat([test_data,test_sex,test_embarked], axis = 1)
train_data.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,male,Q,S
0,1,0,3,22.0,1,0,1,0,1
1,2,1,1,38.0,1,0,0,0,0
2,3,1,3,26.0,0,0,0,0,1
3,4,1,1,35.0,1,0,0,0,1
4,5,0,3,35.0,0,0,1,0,1


In [37]:
train_output = train_data['S']
train_data = train_data.drop(['S'], axis = 1)
test_output = test_data['S']
test_data = test_data.drop(['S'], axis = 1)
train_data.head()

,PassengerId,Pclass,Age,SibSp,Parch,male,Q
0,1,3,22.0,1,0,1,0
1,2,1,38.0,1,0,0,0
2,3,3,26.0,0,0,0,0
3,4,1,35.0,1,0,0,0
4,5,3,35.0,0,0,1,0


In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [39]:
log_model = LogisticRegression(max_iter = 1000)
log_model.fit(train_data,train_output)

LogisticRegression(max_iter=1000)

In [40]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Age          418 non-null    float64
 3   SibSp        418 non-null    int64  
 4   Parch        418 non-null    int64  
 5   male         418 non-null    int64  
 6   Q            418 non-null    int64  
dtypes: float64(1), int64(6)
memory usage: 23.0 KB


In [41]:
predictions = log_model.predict(test_data)
accuracy = accuracy_score(test_output,predictions)
print('the accuracy of the model is', accuracy)

the accuracy of the model is 0.7559808612440191
